In [1]:
# !pip install /kaggle/input/kerasapplications -q
# !pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

In [2]:
!pip install timm

     |████████████████████████████████| 287 kB 2.8 MB/s 


In [3]:
import os

# import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import torch
import gc
import sys
# sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import numpy as np
DEBUG = False
import time
import cv2
import PIL.Image
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import albumentations
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
import timm
from albumentations import *
from albumentations.pytorch import ToTensorV2
device = torch.device('cuda') if not DEBUG else torch.device('cpu')

import tensorflow as tf

import random

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=666)

effnet_pytorch = "../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master"
sys.path.append(effnet_pytorch)
from efficientnet_pytorch import EfficientNet

In [4]:
USAR_CV = False
USAR_CONJUNTO_TEST = True

In [5]:
USAR_PYTORCH = True
USAR_KERAS = True

model_names = ['fold0', 'fold1', 'fold2', 'fold3', 'fold4'] * 5
enet_type  = [ 
#     'efficientB5_512', 'efficientB5_512', 'efficientB5_512', 'efficientB5_512', 'efficientB5_512', 
    'effficientB4-attention', 'effficientB4-attention', 'effficientB4-attention', 'effficientB4-attention', 'effficientB4-attention',
    'resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d',
    'resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320',
    'resnet200d_320_512', 'resnet200d_320_512', 'resnet200d_320_512', 'resnet200d_320_512', 'resnet200d_320_512'
#     'resnet200d_Mejores', 'resnet200d_Mejores', 'resnet200d_Mejores', 'resnet200d_Mejores', 'resnet200d_Mejores'
]
model_path  = [
#         '../input/efficient-b5-chestx/efficientnet_b5_fold0_step0_best_loss_cpu.pth',
#         '../input/efficient-b5-chestx/efficientnet_b5_fold1_step0_best_loss_cpu.pth',
#          '../input/efficient-b5-chestx/efficientnet_b5_fold2_step0_best_loss_cpu.pth',
#          '../input/efficient-b5-chestx/efficientnet_b5_fold3_step0_best_loss_cpu.pth',
#          '../input/efficient-b5-chestx/efficientnet_b5_fold4_step0_best_loss_cpu.pth',
        '../input/efficientb4attention/efficientnet-b4_fold0_step0_best_loss_cpu.pth',
        '../input/efficientb4attention/efficientnet-b4_fold1_step0_best_loss.pth',
        '../input/efficientb4attention/efficientnet-b4_fold2_step0_best_loss_cpu.pth',
        '../input/efficientb4attention/efficientnet-b4_fold3_step0_best_loss.pth',
        '../input/efficientb4attention/efficientnet-b4_fold4_step0_best_loss_cpu.pth',
        '../input/resnext50-32x4d/resnext50_32x4d_fold0_step0_best_loss_cpu_12epochs.pth',
        '../input/resnext50-32x4d/resnext50_32x4d_fold1_step0_best_loss_cpu_12epochs.pth',
        '../input/resnext50-32x4d/resnext50_32x4d_fold2_step0_best_loss_12epochs.pth',
        '../input/resnext50-32x4d/resnext50_32x4d_fold3_step0_best_loss_12epochs.pth',
        '../input/resnext50-32x4d/resnext50_32x4d_fold4_step0_best_loss_12epochs.pth',   
        '../input/resnet200d-640/resnet200d_320_fold0_step0_best_loss_cpu.pth',
        '../input/resnet200d-640/resnet200d_320_fold1_step0_best_loss_cpu.pth',
        '../input/resnet200d-640/resnet200d_320_fold2_step0_best_loss_cpu.pth',
        '../input/resnet200d-640/resnet200d_320_fold3_step0_best_loss_cpu.pth',
        '../input/resnet200d-640/resnet200d_320_fold4_step0_best_loss_cpu.pth',
        '../input/resnet200d-chestx/resnet200d_320_fold0_step0_best_loss_cpu.pth',
        '../input/resnet200d-chestx/resnet200d_320_fold1_step0_best_loss_cpu.pth',
        '../input/resnet200d-chestx/resnet200d_320_fold2_step0_best_loss_cpu.pth',
        '../input/resnet200d-chestx/resnet200d_320_fold3_step0_best_loss_cpu.pth',
        '../input/resnet200d-chestx/resnet200d_320_fold4_step0_best_loss_cpu.pth'
#     '../input/mejores-modelos-encontrados/resnet200d_fold0_cv953.pth/resnet200d_fold0_cv953.pth',
#       '../input/mejores-modelos-encontrados/resnet200d_fold1_cv955.pth/resnet200d_fold1_cv955.pth',
#       '../input/mejores-modelos-encontrados/resnet200d_fold2_cv955.pth/resnet200d_fold2_cv955.pth',
#       '../input/mejores-modelos-encontrados/resnet200d_fold3_cv957.pth/resnet200d_fold3_cv957.pth',
#       '../input/mejores-modelos-encontrados/resnet200d_fold4_cv954.pth/resnet200d_fold4_cv954.pth'
]

In [6]:
# model_names = ['fold0', 'fold1', 'fold2', 'fold3', 'fold4']

# enet_type  = ['resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320']

# model_path = [
#     '../input/resnet200d-640/resnet200d_320_fold0_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold1_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold2_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold3_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold4_step0_best_loss_cpu.pth'
# ]

Carga del test fold
---

In [7]:
TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test'
FOLDS_PATH = '../input/split-k-folds/'
TRAIN_PATH = '../input/ranzcr-clip-catheter-line-classification/train'

target_size=11
target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
             'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
             'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
             'Swan Ganz Catheter Present']
MORE_LAYERS_MODEL = False

BS = 128

# cargamos los DATOS
dfx = pd.read_csv(FOLDS_PATH + "train_folds.csv") 
 
# filtramos. Por defecto dejo en "0" el fold_test
df_test_fold = dfx[dfx.fold_test == 0].reset_index(drop=True)
print(df_test_fold.shape)
df_test_fold['file_path'] = df_test_fold.StudyInstanceUID.apply(lambda x: os.path.join('../input/ranzcr-clip-catheter-line-classification/train', f'{x}.jpg'))

test_targets = df_test_fold[target_cols].values

(1245, 15)


Modelos Pytorch
---

In [8]:
class PAM_Module(nn.Module):
    """ Position attention module"""
    #Ref from SAGAN
    def __init__(self, in_dim):
        super(PAM_Module, self).__init__()
        self.chanel_in = in_dim

        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        """
            inputs :
                x : input feature maps( B X C X H X W)
            returns :
                out : attention value + input feature
                attention: B X (HxW) X (HxW)
        """
        m_batchsize, C, height, width = x.size()
        proj_query = self.query_conv(x).view(m_batchsize, -1, width*height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(m_batchsize, -1, width*height)
        energy = torch.bmm(proj_query, proj_key)
        attention = torch.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(m_batchsize, -1, width*height)

        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma*out + x
        return out


class CAM_Module(nn.Module):
    """ Channel attention module"""
    def __init__(self, in_dim):
        super(CAM_Module, self).__init__()
        self.chanel_in = in_dim
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X H X W)
            returns :
                out : attention value + input feature
                attention: B X C X C
        """
        m_batchsize, C, height, width = x.size()
        proj_query = x.view(m_batchsize, C, -1)
        proj_key = x.view(m_batchsize, C, -1).permute(0, 2, 1)
        energy = torch.bmm(proj_query, proj_key)
        energy_new = torch.max(energy, -1, keepdim=True)[0].expand_as(energy)-energy
        attention = torch.softmax(energy_new, dim=-1)
        proj_value = x.view(m_batchsize, C, -1)

        out = torch.bmm(attention, proj_value)
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma*out + x
        return out


class CBAM(nn.Module):
    def __init__(self, in_channels):
        # def __init__(self):
        super(CBAM, self).__init__()
        inter_channels = in_channels // 4
        self.conv1_c = nn.Sequential(nn.Conv2d(in_channels, inter_channels, 3, padding=1, bias=False),
                                     nn.BatchNorm2d(inter_channels),
                                     nn.ReLU())

        self.conv1_s = nn.Sequential(nn.Conv2d(in_channels, inter_channels, 3, padding=1, bias=False),
                                     nn.BatchNorm2d(inter_channels),
                                     nn.ReLU())

        self.channel_gate = CAM_Module(inter_channels)
        self.spatial_gate = PAM_Module(inter_channels)

        self.conv2_c = nn.Sequential(nn.Conv2d(inter_channels, in_channels, 3, padding=1, bias=False),
                                     nn.BatchNorm2d(in_channels),
                                     nn.ReLU())
        self.conv2_a = nn.Sequential(nn.Conv2d(inter_channels, in_channels, 3, padding=1, bias=False),
                                     nn.BatchNorm2d(in_channels),
                                     nn.ReLU())

    def forward(self, x):
        feat1 = self.conv1_c(x)
        chnl_att = self.channel_gate(feat1)
        chnl_att = self.conv2_c(chnl_att)

        feat2 = self.conv1_s(x)
        spat_att = self.spatial_gate(feat2)
        spat_att = self.conv2_a(spat_att)

        x_out = chnl_att + spat_att

        return x_out



In [9]:
# ====================================================
# MODEL 2
# ====================================================
class CustomResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d_320', MORE_LAYERS_MODEL = False, pretrained=False):
        super().__init__()
        self.use_more_layers = MORE_LAYERS_MODEL
        self.model = timm.create_model(model_name, pretrained=False)
        if pretrained:
            pretrained_path = '../input/resnet200d-pretrained-weight/resnet200d_ra2-bdba9bf9.pth'
            self.model.load_state_dict(torch.load(pretrained_path))
            print(f'load {model_name} pretrained model')
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        if self.use_more_layers:
            self.sigmoid_fc = nn.Linear(in_features=n_features, out_features=512)
            self.bs = nn.BatchNorm1d(num_features = 512)
            self.prelu = nn.PReLU()
            self.fc = nn.Linear(512, 11)
        else:
            self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        if self.use_more_layers:
            output = F.sigmoid(self.sigmoid_fc(pooled_features))
            output = self.bs(output)
            output = self.prelu(output)
            output = self.fc(output) #output pooled_features
        else:
            output = self.fc(pooled_features)
        # features, _, y_preds
        return output
        
########################################################################
########################################################################
########################################################################
class ResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

########################################################################
########################################################################
########################################################################
class EfficientNetB5(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b5_ns'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.classifier.in_features
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.classifier(pooled_features)
        return output
    
    
########################################################################
########################################################################
########################################################################
class EfficientJustAttention(nn.Module):

    def __init__(self, model_name, target_size=11):
        super().__init__()
        self.backbone = EfficientNet.from_name(model_name)

        self.backbone._dropout = nn.Dropout(0.1)
        n_features = self.backbone._fc.in_features
        self.backbone._fc = nn.Linear(n_features, target_size)

        self.local_fe = CBAM(n_features)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Sequential(nn.Linear(n_features + n_features, n_features),
                                        nn.BatchNorm1d(n_features),
                                        nn.Dropout(0.1),
                                        nn.ReLU(),
                                        nn.Linear(n_features, target_size))

    def forward(self, image):
        enc_feas = self.backbone.extract_features(image)

        # use default's global features
        global_feas = self.backbone._avg_pooling(enc_feas)
        global_feas = global_feas.flatten(start_dim=1)
        global_feas = self.dropout(global_feas)

        local_feas = self.local_fe(enc_feas)
        local_feas = torch.sum(local_feas, dim=[2,3])
        local_feas = self.dropout(local_feas)

        all_feas = torch.cat([global_feas, local_feas], dim=1)
        outputs = self.classifier(all_feas)
        return outputs



Datasets
---

In [10]:
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.labels = df[target_cols].values
        self.transform = transform
        self.shape = df.shape

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TRAIN_PATH}/{file_name}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = torch.tensor(self.labels[idx]).float()
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image, labels

In [11]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def get_transforms(IMAGE_SIZE):
        return Compose([
            Resize(IMAGE_SIZE, IMAGE_SIZE),
            Normalize(
                 mean=[0.485, 0.456, 0.406],
             std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
def inference(model, test_loader, device):
    model.eval()
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    probs_tta1 = []
    probs_tta2 = []
    losses = AverageMeter()
    losses_tta1 = AverageMeter()
    losses_tta2 = AverageMeter()
    criterion = torch.nn.BCEWithLogitsLoss()
    
    with torch.no_grad():
        for i, (images, labels) in tk0:
            images = images.to(device)
            labels = labels.to(device)
            batch_size = labels.size(0)
            
            y_preds1 = model(images)
            y_preds2 = model(images.flip(-1))
            y_preds3 = model(images.flip(1))
            
            # aqui calculo el error directamente
            loss = criterion(y_preds1, labels)
            losses.update(loss.item(), batch_size)
            
            loss = criterion((y_preds1 + y_preds2) / 2, labels)
            losses_tta1.update(loss.item(), batch_size)
            
            loss = criterion((y_preds1 + y_preds2 + y_preds3) / 3, labels)
            losses_tta2.update(loss.item(), batch_size)
        
            y_preds = y_preds1.sigmoid().to('cpu').numpy()
            y_preds2 = (y_preds +\
                       y_preds2.sigmoid().to('cpu').numpy()
                      ) / 2
            y_preds3 = (y_preds +\
               y_preds2 +\
               y_preds3.sigmoid().to('cpu').numpy()
              ) / 3
            
            probs.append(y_preds)
            probs_tta1.append(y_preds2)
            probs_tta2.append(y_preds3)
    probs = np.concatenate(probs)
    probs_tta1 = np.concatenate(probs_tta1)
    probs_tta2 = np.concatenate(probs_tta2)
    
    print(losses.avg)
    print(losses_tta1.avg)
    print(losses_tta2.avg)
    tipo_tta = 0
    
    if losses.avg <= losses_tta1.avg and losses.avg <= losses_tta2.avg:
        print("Prediccion pura best loss")
        final_probs = probs
        final_loss = losses.avg
        tipo_tta = 0
    elif losses_tta1.avg <= losses.avg and losses_tta1.avg <= losses_tta2.avg:
        print("Prediccion 1 tta best loss")
        final_probs = probs_tta1
        final_loss = losses_tta1.avg
        tipo_tta = 1
    elif losses_tta2.avg <= losses.avg and losses_tta2.avg <= losses_tta1.avg:
        print("Prediccion 2 tta best loss")
        final_probs = probs_tta2
        final_loss = losses_tta2.avg
        tipo_tta = 2
    return tipo_tta, final_loss, final_probs

def get_auc(model,
            test_loader_pytorch, device):
        
    # ******** pytorch ***************
    tipo_tta, loss_avg, y_preds = inference(model, test_loader_pytorch, device)
    # predict 
    y_real = df_test_fold.iloc[:, 1:13]
    
    # get AUC
    acc_fold = np.mean([roc_auc_score(y_real.iloc[:, i], y_preds[:, i]) for i in range(11)])
    print("Final ACC:", acc_fold)
    
#     a = torch.from_numpy(y_preds).sigmoid()
#     b = torch.from_numpy(y_real.iloc[:, 0:11].values)
#     b = b.type_as(a)

#     criterion = torch.nn.BCEWithLogitsLoss()
#     loss = criterion(a, b)
#     print('Loss: ', loss) 
    print('Loss: ', loss_avg) 
        
    return tipo_tta, loss_avg, y_preds

******** PYTORCH ********
---

In [12]:
def save_csv(df_original, preds, name):
    df = df_test_fold.copy()
    df[target_cols] = preds
    
    df.to_csv(name + '.csv', index=False)
    print(name + ' saved as csv')
    
def get_preds_pytorch():
    
    # los test_loaders
    test_dataset = TrainDataset(df_test_fold, transform=get_transforms(512))
    test_loader_512 = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False,  num_workers=4 , pin_memory=True)
    
    test_dataset = TrainDataset(df_test_fold, transform=get_transforms(640))
    test_loader_640 = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False,  num_workers=4 , pin_memory=True)
    
    # inferencias
    test_preds = []
    losses = []
    names = []
    for i in range(len(enet_type)):
       # 5 fold resnet
        if enet_type[i] == 'resnet200d' or enet_type[i] == 'resnet200d_Mejores':
            print('resnet200d loaded')
            model = ResNet200D()
            model = model.to(device)
            if not DEBUG:
                model.load_state_dict(torch.load(model_path[i], map_location='cuda:0'))
            else:
                model.load_state_dict(torch.load(model_path[i], map_location=torch.device('cpu')))
#                 if tta:
#                     test_preds += [tta_inference_func(test_loader_512)]
#                 else:
#                     test_preds += [inference_func(test_loader_512)]
            tipo_tta, loss, predictions = get_auc(model, test_loader_512, device)
            test_preds += [predictions]
        # 1 fold resnet200_320
        if enet_type[i] == 'resnet200d_320':
            print('resnet200d_320 loaded')
            model = ResNet200D()
            model = model.to(device)
            model.load_state_dict(torch.load(model_path[i], map_location='cuda:0')['model'])
            # preds
            tipo_tta, loss, predictions = get_auc(model, test_loader_640, device)
            test_preds += [predictions]
        # 1 fold efficientB5
        if enet_type[i] == 'efficientB5':
            print('efficientB5 loaded')
            model = EfficientNetB5()
            model = model.to(device)
            model.load_state_dict(torch.load(model_path[i], map_location='cuda:0')['model'])
            # preds
            tipo_tta, loss, predictions = get_auc(model, test_loader_640, device)
            test_preds += [predictions]

        # 5 fold resnext50_32x4d
        if enet_type[i] == 'resnext50_32x4d':
            print('resnext50_32x4d loaded')
            model = CustomResNet200D(enet_type[i])
            model = model.to(device)
            model.load_state_dict(torch.load(model_path[i], map_location='cuda:0')['model'])
            # preds
            tipo_tta, loss, predictions = get_auc(model, test_loader_640, device)
            test_preds += [predictions]
        # 5 fold resnet200_320_512
        if enet_type[i] == 'resnet200d_320_512':
            print('resnet200d_320_512 loaded')
            model = ResNet200D()
            model = model.to(device)
            model.load_state_dict(torch.load(model_path[i], map_location='cuda:0')['model'])
            # preds
            tipo_tta, loss, predictions = get_auc(model, test_loader_512, device)
            test_preds += [predictions]
        # 5 fold efficientB5_512
        if enet_type[i] == 'efficientB5_512':
            print('efficientB5_512 loaded')
            model = EfficientNetB5()
            model = model.to(device)
            model.load_state_dict(torch.load(model_path[i], map_location='cuda:0')['model'])
            # preds
            tipo_tta, loss, predictions = get_auc(model, test_loader_512, device)
            test_preds += [predictions]
        # attention efficientb4
        if enet_type[i] == 'effficientB4-attention':
            print('efficientB4-attention loaded')
            model = EfficientJustAttention('efficientnet-b4')
            model = model.to(device)
            model.load_state_dict(torch.load(model_path[i], map_location='cuda:0')['model'])
            # preds
            tipo_tta, loss, predictions = get_auc(model, test_loader_512, device)
            test_preds += [predictions]
        
        losses.append(loss)
        names.append(enet_type[i] + "_" + model_names[i])
        # save
        save_csv(df_test_fold, predictions, enet_type[i] + "_" + model_names[i])
        # clean
        del model
        gc.collect()
        torch.cuda.empty_cache()
        print('--------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------')
    del test_loader_640, test_loader_512
    
    #save csv with the losses
    df_model_losses = pd.DataFrame(
    {'name': names,
     'loss': losses
    })
    df_model_losses.to_csv('models_losses.csv', index=False)
    
    return np.mean(test_preds, axis=0) 

In [13]:
if USAR_PYTORCH:
    
    get_preds_pytorch()
    gc.collect()
    torch.cuda.empty_cache()


efficientB4-attention loaded


  0%|          | 0/20 [00:00<?, ?it/s]

0.16530177554931028
0.1622647337884788
0.1751492670262195
Prediccion 1 tta best loss
Final ACC: 0.9293013410808553
Loss:  0.1622647337884788
effficientB4-attention_fold0 saved as csv
--------------------------------------------------------------------------
--------------------------------------------------------------------------
--------------------------------------------------------------------------
efficientB4-attention loaded


  0%|          | 0/20 [00:00<?, ?it/s]

0.16028010610595764
0.15841961663410845
0.1641607790826315
Prediccion 1 tta best loss
Final ACC: 0.9295763353145816
Loss:  0.15841961663410845
effficientB4-attention_fold1 saved as csv
--------------------------------------------------------------------------
--------------------------------------------------------------------------
--------------------------------------------------------------------------
efficientB4-attention loaded


...

0.13454056051840266
0.1305974316525172
0.13139487106397926
Prediccion 1 tta best loss
Final ACC: 0.953447568051093
Loss:  0.1305974316525172
resnet200d_320_512_fold2 saved as csv
--------------------------------------------------------------------------
--------------------------------------------------------------------------
--------------------------------------------------------------------------
resnet200d_320_512 loaded


  0%|          | 0/20 [00:00<?, ?it/s]

0.13712203727668548
0.13500725309772185
0.13380976875862444
Prediccion 2 tta best loss
Final ACC: 0.9534776280941144
Loss:  0.13380976875862444
resnet200d_320_512_fold3 saved as csv
--------------------------------------------------------------------------
--------------------------------------------------------------------------
--------------------------------------------------------------------------
resnet200d_320_512 loaded


  0%|          | 0/20 [00:00<?, ?it/s]

0.1347143872076249
0.13281487686088286
0.1322868738428177
Prediccion 2 tta best loss
Final ACC: 0.9554723572142148
Loss:  0.1322868738428177
resnet200d_320_512_fold4 saved as csv
--------------------------------------------------------------------------
--------------------------------------------------------------------------
--------------------------------------------------------------------------


KERAS
---

In [14]:
model_names = ['fold0', 'fold1', 'fold2', 'fold3', 'fold4'] * 4
enet_type_keras  = [ 
    'inceptionV2_640', 'inceptionV2_640', 'inceptionV2_640', 'inceptionV2_640', 'inceptionV2_640',
    'efficient-B7', 'efficient-B7', 'efficient-B7', 'efficient-B7', 'efficient-B7',
    'xception_840','xception_840','xception_840','xception_840','xception_840',
    'efficientnet-B5', 'efficientnet-B5', 'efficientnet-B5', 'efficientnet-B5', 'efficientnet-B5'
]

model_path = [
    '../input/inceptionresnetv2-ranzcr/model_fold0_InceptionResNetV2.h5',
    '../input/inceptionresnetv2-ranzcr/model_fold1_InceptionResNetV2.h5',
    '../input/inceptionresnetv2-ranzcr/model_fold2_InceptionResNetV2.h5',
    '../input/inceptionresnetv2-ranzcr/model_fold3_InceptionResNetV2.h5',
    '../input/inceptionresnetv2-ranzcr/model_fold4_InceptionResNetV2.h5',
    '../input/efficientnetb7v2/model_fold0efficient.h5',
     '../input/efficientnetb7v2/model_fold1efficient.h5',
     '../input/efficientnetb7v2/model_fold2efficient.h5',
     '../input/efficientnetb7v2/model_fold3efficient.h5',
     '../input/efficientnetb7v2/model_fold4efficient.h5',
    '../input/xceptionv2/model_fold0efficient.h5',
    '../input/xceptionv2/model_fold1efficient.h5',
    '../input/xceptionv2/model_fold2efficient.h5',
    '../input/xceptionv2/model_fold3efficient.h5',
    '../input/xceptionv2/model_fold4efficient.h5',
    '../input/efficientb5v2/model_fold0efficient.h5',
    '../input/efficientb5v2/model_fold1efficient.h5',
    '../input/efficientb5v2/model_fold2efficient.h5',
    '../input/efficientb5v2/model_fold3efficient.h5',
    '../input/efficientb5v2/model_fold4efficient.h5',
]

In [15]:
tta_types = [0, 1] # 0 es ningun tta
BATCH_SIZE = 64

In [16]:
if USAR_KERAS:
    !pip install /kaggle/input/kerasapplications -q
    
    !pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

In [17]:
if USAR_KERAS:
    from efficientnet.tfkeras import EfficientNetB7

In [18]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(lotpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            # Decode a PNG-encoded image to a uint8 or uint16 tensor
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            # Decode a JPEG-encoded image to a uint8 tensor
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        # Casts a tensor to the type float32 and divides by 255
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True, augment_type = 0):
    def augment(img):
        if (augment_type == 1):
            img = tf.image.random_flip_left_right(img) # Flip an image horizontally (left to right). -->MEJORA
        
        if (augment_type == 2):
            img = tf.image.random_flip_up_down(img)
        # added with xception
        #img = tf.image.random_saturation(img, 0.8, 1.2)
        #img = tf.image.random_brightness(img, 0.1)
        #img = tf.image.random_contrast(img, 0.9, 1.2)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir="", 
                  aug_type = 0):
    # aug_type: 
    # 0: none
    # 1: random_flip_left_right
    # 2: random_flip_up_down
    
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None, augment_type = aug_type)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [19]:
if USAR_KERAS:
    COMPETITION_NAME = "ranzcr-clip-catheter-line-classification"
    strategy = auto_select_accelerator()
    BATCH_SIZE = strategy.num_replicas_in_sync * 16
    
    IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600)


Running on 1 replicas


In [20]:
def get_loss(y_real, y_pred):
    
    bce = tf.keras.losses.BinaryCrossentropy(from_logits = True)
    loss = np.mean([bce(y_real.iloc[:, i], y_pred[:, i]) for i in range(11)])
    
    return loss

In [21]:
def inference(model, test_decoder):

    preds = []
    probs = []
    probs_tta1 = []
    y_real = df_test_fold.iloc[:, 1:13]
    # original
    dtest_tta = build_dataset(df_test_fold.file_path, df_test_fold[target_cols].values,
                              bsize=BATCH_SIZE, repeat=False, shuffle=False,
                              augment=True, cache=False,
                              decode_fn=test_decoder,
                              aug_type = 0)
    metrics = model.evaluate(dtest_tta, verbose=1)
    # get loss original
    y_preds = model.predict(dtest_tta, verbose=1)
    loss = get_loss(y_real, y_preds)
    preds.append(y_preds)
    
    # 1 tta
    dtest_tta = build_dataset(df_test_fold.file_path, df_test_fold[target_cols].values,
                              bsize=BATCH_SIZE, repeat=False, shuffle=False,
                          augment=True, cache=False,
                          decode_fn=test_decoder,
                          aug_type = 1)
    metrics_tta1 = model.evaluate(dtest_tta, verbose=1)
    y_preds_tta1 = model.predict(dtest_tta, verbose=1)
    preds.append(y_preds_tta1)
    # get loss original + 1 tta
    y_preds_tta1 = np.mean(preds, axis = 0)
    loss_tta1 = get_loss(y_real, y_preds_tta1)
        
    print(metrics[0])
    print(metrics[1])
    tipo_tta = 0  
    
    if loss <= loss_tta1:
        print("Prediccion pura best loss")
        final_probs = y_preds
        final_loss = loss
        tipo_tta = 0
    elif loss_tta1 <= loss:
        print("Prediccion 1 tta best loss")
        final_probs = y_preds_tta1
        final_loss = loss_tta1
        tipo_tta = 1
        
    return tipo_tta, final_loss, final_probs

def get_auc(model,
            test_decoder):
        
    # ******** pytorch ***************
    tipo_tta, loss_avg, y_preds = inference(model, test_decoder)
    # predict 
    y_real = df_test_fold.iloc[:, 1:13]
    
    # get AUC
    acc_fold = np.mean([roc_auc_score(y_real.iloc[:, i], y_preds[:, i]) for i in range(11)])
    print("Final ACC:", acc_fold)
    
#     a = torch.from_numpy(y_preds).sigmoid()
#     b = torch.from_numpy(y_real.iloc[:, 0:11].values)
#     b = b.type_as(a)

#     criterion = torch.nn.BCEWithLogitsLoss()
#     loss = criterion(a, b)
#     print('Loss: ', loss) 
    print('Loss: ', loss_avg) 
        
    return tipo_tta, loss_avg, y_preds

In [22]:
if USAR_KERAS:
    def get_preds_keras():
#         dtest = build_dataset(
#             df_test_fold.file_path, bsize=128, repeat=False, 
#             shuffle=False, augment=False, cache=False,
#             decode_fn=test_decoder_600
#         )
        
        # inferencias
        test_preds = []
        losses = []
        names = []
        with strategy.scope():
            for i in range(len(enet_type_keras)):
               # 5 fold resnet
                if enet_type_keras[i] == 'efficient-B7':
                    test_decoder_600 = build_decoder(with_labels=True, target_size=(600, 600))
                    print('efficientnet-B7 loaded')
                    model = tf.keras.models.load_model(model_path[i])
                    # preds
                    tipo_tta, loss, predictions = get_auc(model, test_decoder_600)
                    test_preds += [predictions]
                    del test_decoder_600
                    
                    losses.append(loss)
                elif enet_type_keras[i] == 'inceptionV2_640':
                    test_decoder_640 = build_decoder(with_labels=True, target_size=(640, 640))
                    print('InceptionV2 loaded')
                    model = tf.keras.models.load_model(model_path[i])
                    # preds
                    tipo_tta, loss, predictions = get_auc(model, test_decoder_640)
                    test_preds += [predictions]
                    del test_decoder_640
                    
                    losses.append(loss)
                elif enet_type_keras[i] == 'xception_840':
                    test_decoder_840 = build_decoder(with_labels=True, target_size=(840, 840))
                    print('Xception loaded')
                    model = tf.keras.models.load_model(model_path[i])
                    # preds
                    tipo_tta, loss, predictions = get_auc(model, test_decoder_840)
                    test_preds += [predictions]
                    del test_decoder_840
                elif enet_type_keras[i] == 'efficientnet-B5':
                    test_decoder_840 = build_decoder(with_labels=True, target_size=(840, 840))
                    print('Xception loaded')
                    model = tf.keras.models.load_model(model_path[i])
                    # preds
                    tipo_tta, loss, predictions = get_auc(model, test_decoder_840)
                    test_preds += [predictions]
                    del test_decoder_840
                    
                names.append(enet_type_keras[i] + "_" + model_names[i])
                # save
                save_csv(df_test_fold, predictions, enet_type_keras[i] + "_" + model_names[i])
                # clean
                del model
                gc.collect()
                print('--------------------------------------------------------------------------')
                print('--------------------------------------------------------------------------')
                print('--------------------------------------------------------------------------')

In [23]:
# test_decoder_600 = build_decoder(with_labels=True, target_size=(640, 640))
# dtest_tta = build_dataset(df_test_fold.file_path, df_test_fold[target_cols].values,
#                           bsize=BATCH_SIZE, repeat=False, shuffle=False,
#                           augment=True, cache=False,
#                           decode_fn=test_decoder_600,
#                           aug_type = 0)
# model = tf.keras.models.load_model(model_path[0])
# a = model.evaluate(dtest_tta, verbose=1)
# y_preds = model.predict(dtest_tta, verbose=1)
# a

In [24]:
# get_loss(df_test_fold.iloc[:, 1:13], y_preds)

In [25]:
if USAR_KERAS:
    get_preds_keras()

InceptionV2 loaded
78/78 [==============================] - 115s 1s/step
0.17441414296627045
0.9089744091033936
Prediccion 1 tta best loss
Final ACC: 0.9140080831548982
Loss:  0.6782217
inceptionV2_640_fold0 saved as csv
--------------------------------------------------------------------------
--------------------------------------------------------------------------
--------------------------------------------------------------------------
InceptionV2 loaded
78/78 [==============================] - 119s 2s/step
0.16511255502700806
0.9086307883262634
Prediccion 1 tta best loss
Final ACC: 0.9199390186004369
Loss:  0.6752681
inceptionV2_640_fold1 saved as csv
--------------------------------------------------------------------------
--------------------------------------------------------------------------
--------------------------------------------------------------------------
InceptionV2 loaded
78/78 [==============================] - 118s 2s/step
0.16339609026908875
0.9228323101997